In [1]:
import nltk

Replacer module

In [2]:
import re
from nltk.corpus import wordnet
R_patterns = [
   (r'won\'t', 'will not'),
   (r'can\'t', 'cannot'),
   (r'i\'m', 'i am'),
   (r'(\w+)\'ll', '\g<1> will'),
   (r'(\w+)n\'t', '\g<1> not'),
   (r'(\w+)\'ve', '\g<1> have'),
   (r'(\w+)\'s', '\g<1> is'),
   (r'(\w+)\'re', '\g<1> are'),
]
class REReplacer(object):
    def __init__(self, patterns=R_patterns):
       self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
       s = text
       for (pattern, repl) in self.patterns:
          s = re.sub(pattern, repl, s)
       return s

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import string
import nltk ,re
import math
import numpy as np
from nltk.corpus import stopwords

critic_value = float(input("Enter critical value:"))

Enter critical value:2.57


In [5]:
stop_words = set(stopwords.words('english'))

Importing the data

In [6]:
file = open("text.txt","r")
text = file.read()

In [7]:
print(text)

she knocked the door
they knocked at the door
a man knocked on the metal front door
she knocked door


Sentence Tokenization


In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
sents = nltk.sent_tokenize(text)

Punctuation Removal

In [11]:
sents_1 = []
rep_word = REReplacer()
for i in sents:
    n = rep_word.replace(i)
    n = n.translate(str.maketrans('', '', string.punctuation))
    sents_1.append(n)

In [12]:
sents_1

['she knocked the door\nthey knocked at the door\na man knocked on the metal front door\nshe knocked door']

Word Tokenization

In [13]:
ws = []
for j in sents_1:
    sent = nltk.word_tokenize(j)
    for s in sent:
        ws.append(s)

Stopwords Removal

In [14]:
texts = []
for w in ws:
    if w not in stop_words:
        texts.append(w)

In [15]:
print(texts)
N = len(texts)

['knocked', 'door', 'knocked', 'door', 'man', 'knocked', 'metal', 'front', 'door', 'knocked', 'door']


Finding Frequency and Probability of a word

In [16]:
vocabulary = []
for word in texts :
    if word not in vocabulary:
        vocabulary.append(word)
voc_len = len(vocabulary)

In [17]:
print(vocabulary)

['knocked', 'door', 'man', 'metal', 'front']


Preparation for t-test and chi^2-test

In [18]:
print("Unique words:",voc_len)
print("Total words:",N)

Unique words: 5
Total words: 11


T-test

In [19]:
def T_test(w1,w2,c1,c2,c12):

    p_w1 = c1/N
    p_w2 = c2/N

    exp_mean = p_w1*p_w2 #expected mean

    ob_mean = c12/N #observed mean
    s_2 = ob_mean #s^2
    diff =  ob_mean - exp_mean
    v = math.sqrt((s_2/N))
    t = diff/v
    t = round(t, 2)
    d = {}
    d['t-value'] = t
    d['word1'] = w1
    d['word2'] = w2
    return d

chi^2 test

In [20]:
def chi_2_test(w1,w2,c1,c2,c12):
    a = c12
    b = c1 - a
    cx = c2 - a
    r2 = N - c1
    d = r2 - cx
    r1 = a+b
    e = a+cx
    f = b+d

    e_w1 = (r1*e)/N
    e_w2 = (r1*f)/N
    e_w3 = (e*r2)/N
    e_w4 = (f*r2)/N
    sum1 = (np.square(a-e_w1))/e_w1
    sum2 = (np.square(b-e_w2))/e_w2
    sum3 = (np.square(cx-e_w3))/e_w3
    sum4 = (np.square(d-e_w4))/e_w4


    c = {}
    c['chi-value'] = sum1 + sum2 + sum3 + sum4
    c['word1'] = w1
    c['word2'] = w2
    return c

In [21]:
ttest = []
chi2test=[]
for i in range(len(texts)-1):
    w1=texts[i]
    w2=texts[i+1]
    w12 = w1 +" "+ w2
    c1 = 0
    c2 = 0
    c12 = 0
    for w in range(len(texts)-1):
        if texts[w] == w1:
            c1 = c1+1
            if texts[w+1] == w2:
                c12 = c12+1
        if texts[w] == w2:
            c2 = c2+1
    ttest.append(T_test(w1,w2,c1,c2,c12))
    chi2test.append(chi_2_test(w1,w2,c1,c2,c12))

In [22]:
print("t-test:\n")
for d in ttest:
    if d['t-value'] > critic_value:
        print("t-value:",d['t-value'],"\t Words: ",d['word1'],"and",d['word2']," are collocated!",end = "\n")
    else:
        print("t-value:",d['t-value'],"\t Words: ",d['word1'],"and",d['word2'], " are not collocated!",end = "\n")

t-test:

t-value: 1.1 	 Words:  knocked and door  are not collocated!
t-value: 0.64 	 Words:  door and knocked  are not collocated!
t-value: 1.1 	 Words:  knocked and door  are not collocated!
t-value: 0.73 	 Words:  door and man  are not collocated!
t-value: 0.64 	 Words:  man and knocked  are not collocated!
t-value: 0.64 	 Words:  knocked and metal  are not collocated!
t-value: 0.91 	 Words:  metal and front  are not collocated!
t-value: 0.73 	 Words:  front and door  are not collocated!
t-value: 0.64 	 Words:  door and knocked  are not collocated!
t-value: 1.1 	 Words:  knocked and door  are not collocated!


In [23]:
print("chi^2-test:\n")
for d in chi2test:
    if d['chi-value'] > critic_value:
        print("chi-value:",d['chi-value'],"\t Words: ",d['word1'],"and",d['word2']," are collocated!",end = "\n")
    else:
        print("chi-value:",d['chi-value'],"\t Words: ",d['word1'],"and",d['word2'], " are not collocated!",end = "\n")

chi^2-test:

chi-value: 7.218750000000001 	 Words:  knocked and door  are collocated!
chi-value: 1.6369047619047623 	 Words:  door and knocked  are not collocated!
chi-value: 7.218750000000001 	 Words:  knocked and door  are collocated!
chi-value: 2.933333333333333 	 Words:  door and man  are collocated!
chi-value: 1.925 	 Words:  man and knocked  are not collocated!
chi-value: 1.925 	 Words:  knocked and metal  are not collocated!
chi-value: 10.999999999999996 	 Words:  metal and front  are collocated!
chi-value: 2.9333333333333336 	 Words:  front and door  are collocated!
chi-value: 1.6369047619047623 	 Words:  door and knocked  are not collocated!
chi-value: 7.218750000000001 	 Words:  knocked and door  are collocated!
